In [1]:
import tweepy                   # Python wrapper around Twitter API
import json
import pandas as pd
import csv
from datetime import date
from datetime import datetime
import time
import matplotlib.pyplot as plt

In [4]:
dictionary = {
    "API_key": "eOqbspUqIoWoR199s9J5CDQgY",
    "API_secret_Key": "wwCa03iVNoJq0AfXJ9ssLYaHNECuE65KfEfmTUkkgsJo7ZEd29",
    "Access_Token": "887014753520013313-7JwAu8LysbJsdUxhoiLgGSvqPGbmX2f",
    "Acess_Token_Secret": "X08GHJ4RorHILgwpySYzNyITvvKuxUwEkJbZVn1FX17NY",
    "Bearer_Token" : "AAAAAAAAAAAAAAAAAAAAAF53gAEAAAAAjC18Ngcn%2FjxoHNEOjTf9YDiSQtQ%3DFKouXUO9JHIBtIRFz3X0rq1ts8qNXUabuCd7OYh5s466Hr5eb1"
    
}
 
with open("Secret.json", "w") as outfile:
    json.dump(dictionary, outfile)

In [16]:
secrets = json.loads(open('Secret.json').read())  
consumer_key = secrets['API_key']
consumer_secret = secrets['API_secret_Key']
access_token = secrets['Access_Token']
access_token_secret = secrets['Acess_Token_Secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)  
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)


RT @mauriciomacri: San Juan quiere reponer en las próximas elecciones la Ley de Lemas, que es una vuelta al pasado y un fraude a la represe…
[NUEVO] Consejos para ahorrar energía eléctrica en el hogar

💡Te contamos algunos tips en la gráfica y podés leer m… https://t.co/qmMyco6kKU
Mañana 20.00 hs. en mi querida UCA donde estudie mi carrera de economista https://t.co/Qx1YQ9zQbd
La cantidad de empresas valiosas que gastan horas y horas en temas burocráticos, en vez de usar ese tiempo en hacer… https://t.co/zM2hTizwqb
📢 Last Call!
Orange County Polo Club in Silverado, CA, is pleased to be hosting the USPA Women's Arena Open in the… https://t.co/2DbbBsWLk6
Ernesto! Ernesto! Toma tu huesito y haz tu gracia! https://t.co/yz3g5Yo70A
[NUEVO] Acuerdo con el FMI: cómo le fue al Gobierno con las metas del segundo trimestre.

En el segundo trimestre,… https://t.co/oeq7yaapvZ
RT @Apuntes_: Ahora es el momento de implementar la guardia cívica en el domicilio del fiscal Luciani.
ICYMI: @federalreserv

In [11]:
import json 

status = public_tweets[0]

#convert to string
json_str = json.dumps(status._json)

#deserialise string into python object
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))


{
    "contributors": null,
    "coordinates": null,
    "created_at": "Thu Aug 25 19:36:02 +0000 2022",
    "entities": {
        "hashtags": [],
        "symbols": [],
        "urls": [
            {
                "display_url": "twitter.com/i/web/status/1\u2026",
                "expanded_url": "https://twitter.com/i/web/status/1562886539666792450",
                "indices": [
                    117,
                    140
                ],
                "url": "https://t.co/qmMyco6kKU"
            }
        ],
        "user_mentions": []
    },
    "favorite_count": 2,
    "favorited": false,
    "geo": null,
    "id": 1562886539666792450,
    "id_str": "1562886539666792450",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "is_quote_status": false,
    "lang": "es",
    "place": null,
    "possibly_sensitive": false,
    "possibly_sensitiv

In [12]:
class TweetMiner(object):
    secrets = json.loads(open('Secret.json').read())  

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        secrets['API_key'],
        'consumer_secret':     secrets['API_secret_Key'],
        'access_token_key':    secrets['Access_Token'],
        'access_token_secret': secrets['Acess_Token_Secret']
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="elonmusk", #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

In [13]:

import tweepy
import datetime
import pandas as pd
miner=TweetMiner(result_limit = 200 )

In [14]:
mined_tweets = miner.mine_user_tweets(user='dril', max_pages=17)

mined_tweets_df= pd.DataFrame(mined_tweets)

Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets
Unexpected parameter: include_retweets


In [15]:
mined_tweets_df

,tweet_id,name,screen_name,retweet_count,text,mined_at,created_at,favourite_count,hashtags,status_count,location,source_device,retweet_text,quote_text,quote_screen_name
0,1561830912857219072,wint,dril,410,"""what if among us was squid game"" - DRIL https...",2022-08-25 16:47:30.804016,2022-08-22 21:41:21+00:00,4547,[],11787,None,Twitter Web App,None,None,None
1,1561769888380727296,wint,dril,641,they are letting people vomit in the aisle s o...,2022-08-25 16:47:30.804046,2022-08-22 17:38:52+00:00,7657,[],11787,None,Twitter Web App,None,None,None
2,1561331917273776129,wint,dril,257,RT @fucking_ass: fuckinn//,2022-08-25 16:47:30.804051,2022-08-21 12:38:31+00:00,0,[],11787,None,Twitter Web App,fuckinn//,None,None
3,1561032906826362880,wint,dril,691,fuck my ass hole https://t.co/J2wbHKmgkY,2022-08-25 16:47:30.804055,2022-08-20 16:50:22+00:00,15491,[],11787,None,Twitter for Android,None,None,None
4,1559585315894874112,wint,dril,2506,"it Sucks, its bullshit, its stupid, but you Lo...",2022-08-25 16:47:30.804057,2022-08-16 16:58:09+00:00,13790,[],11787,None,Twitter Web App,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3224,1070556106475266048,wint,dril,404,if youre one of thoe waiters who carries the l...,2022-08-25 16:47:49.427578,2018-12-06 05:50:40+00:00,5269,[],11787,None,Twitter Web Client,None,None,None
3225,1070156456672944133,wint,dril,2163,i like to believe that getting my ass kicked 1...,2022-08-25 16:47:49.427589,2018-12-05 03:22:36+00:00,14608,[],11787,None,Twitter Web Client,None,None,None
3226,1070155635675750400,wint,dril,1767,taking my 3d printed wife to the Arboretum .,2022-08-25 16:47:49.427600,2018-12-05 03:19:20+00:00,13167,[],11787,None,Twitter Web Client,None,None,None
3227,1070086133319487488,wint,dril,7,@Carlos_Lugar @ReesesPuffDaddy give me the 200000,2022-08-25 16:47:49.427612,2018-12-04 22:43:10+00:00,44,[],11787,None,Twitter Web Client,None,None,None


The last thing to remember is there is a limit on API calls, for a standard user this is 900 requests every 15 minutes. Don’t worry, one tweet isn’t one request, but it might be difficult to know when you’re about to go over. For making multiple calls at once try this: